<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/3_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.

### 在同一連接埠下並行存在多個頁面，並且其中一個頁面（page1）需要根據使用者的選擇更新參數並將這些參數儲存到檔案中，你可以採用以下策略：

1. **使用Dash创建多页面应用**：Dash 支持创建多页面应用，每个页面作为一个模块独立存在。你可以通过在应用的根目录下创建一个pages目录，并在其中为每个页面创建一个独立的Python文件来组织你的代码。

2. **页面间的数据传递**：对于page1的参数选择，可以在用户更新选择后，通过Dash的回调机制触发参数的保存

3. **保存参数到文件**：对于参数的保存格式，CSV格式是一个较好的选择，因为它既可以方便地被人阅读，也可以容易地被程序处理。CSV文件能够很好地处理“有多个值或没有值”的情况，你可以通过使用特定的分隔符（例如逗号）来区分不同的值，而空值可以直接留空。

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import pandas as pd
import os

# 初始化Dash应用
app = dash.Dash(__name__)

# 页面1的布局，包含几个输入框和一个提交按钮
page1_layout = html.Div([
    html.H1('页面1'),
    dcc.Input(id='input-1', type='text', placeholder='输入1'),
    dcc.Input(id='input-2', type='text', placeholder='输入2', value=''),
    html.Button('提交', id='submit-val', n_clicks=0),
    html.Div(id='container-button-basic')
])

# 定义应用的初始布局
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# 定义页面路由
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page1':
        return page1_layout
    else:
        return '404'

# 处理页面1表单提交
@app.callback(Output('container-button-basic', 'children'),
              [Input('submit-val', 'n_clicks')],
              [State('input-1', 'value'),
               State('input-2', 'value')])
def update_output(n_clicks, input1, input2):
    if n_clicks > 0:
        # 保存数据到CSV
        data = {'字段1': [input1], '字段2': [input2]}
        df = pd.DataFrame(data)
        df.to_csv('parameters.csv', index=False, encoding='utf-8')
        return '已提交并保存数据'
    return '请提交数据'

if __name__ == '__main__':
    app.run_server(debug=True)


这个例子中，我们有一个页面（page1）包含两个输入框，用户可以输入参数。当用户点击提交按钮时，页面的数据会被读取并保存到一个名为parameters.csv的文件中。每次点击提交，旧的CSV文件都会被新数据覆盖